#Differentiating Flower With CNNs

##Importing Packages

In [0]:
import os
import numpy as np
import glob
import shutil

import tensorflow as tf

import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

##Data Loading

In [0]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL,
                                   fname="flower_photos.tgz",
                                   extract=True)

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

In [0]:
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

In [0]:
for cl in classes:
  img_path = os.path.join(base_dir, cl)
  images = glob.glob(img_path + '/*.jpg')
  print("{}: {} Images".format(cl, len(images)))
  train, val = images[:round(len(images)*0.8)], images[round(len(images)*0.8):]

  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):
      os.makedirs(os.path.join(base_dir, 'train', cl))
    shutil.move(t, os.path.join(base_dir, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', cl)):
      os.makedirs(os.path.join(base_dir, 'val', cl))
    shutil.move(v, os.path.join(base_dir, 'val', cl))

In [0]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

##Data Argumetation

In [0]:
batch_size =100
IMG_SHAPE = 150

In [0]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()


###Random Horizontal flip

In [0]:
image_gen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
)

train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               shuffle=True,
                                               class_mode='binary',
                                               directory=train_dir,
                                               target_size=(IMG_SHAPE,IMG_SHAPE)
                                               )

In [0]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

###Random Rotation

In [0]:
image_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45
)

train_data_gen = image_gen.flow_from_directory(class_mode='binary',
                                               target_size=(IMG_SHAPE,IMG_SHAPE),
                                               batch_size=batch_size,
                                               shuffle=True,
                                               directory=train_dir)

In [0]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

###Apply Zoom

In [0]:
image_gen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.5
)

train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               target_size=(IMG_SHAPE,IMG_SHAPE),
                                               shuffle=True,
                                               directory=train_dir,
                                               class_mode='binary')

In [0]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

###All Together

In [0]:
image_gen_train = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    zoom_range=0.5,
    rotation_range=45,
    height_shift_range=0.3,
    width_shift_range=0.3,
    shear_range=0.3,
    fill_mode='nearest'
)


train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               target_size=(IMG_SHAPE,IMG_SHAPE),
                                               shuffle=True,
                                               directory=train_dir,
                                               class_mode='binary')

In [0]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

###Validation set

In [0]:
image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               target_size=(IMG_SHAPE,IMG_SHAPE),
                                               directory=train_dir,
                                               class_mode='binary')

##Creating Model

####Creating Layers

In [0]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512,activation='relu'),
                                    tf.keras.layers.Dense(5,activation='softmax')
                                    ])

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

###Training Model

In [0]:
epochs = 80

history = model.fit_generator(train_data_gen,
                              epochs=epochs,
                              validation_data=val_data_gen,
                              steps_per_epoch=int(np.ceil(train_data_gen.n / float(batch_size))),
                              validation_steps=int(np.ceil(val_data_gen.n / float(batch_size))))

##Result

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)


plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()